<a href="https://colab.research.google.com/github/Anushka31raut/Practicalandlab/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%


Ans 1  Increasing the Percentage of Validation Set:

Pros:
Typically, as we increase the percentage of the validation set, we are allocating more data for model validation and less for training. This can lead to a more reliable estimate of our model's performance because it is evaluated on a larger and more representative portion of the data.It can help detect overfitting more effectively since the model has less data to fit on during training.

Cons:
With a larger validation set, we have fewer data points available for training. This may result in a model that underfits the training data, leading to lower training accuracy.In some cases, we may not have a large enough dataset to allocate a significant percentage to the validation set without severely limiting the training data, which can hinder model training.

Reducing the Percentage of Validation Set:

Pros:
A smaller validation set means we have more data available for training, which can potentially lead to a more accurate model. The model can learn more from the data and might achieve higher training accuracy.With more training data, it might be possible to train more complex models or use more sophisticated algorithms.

Cons:
A smaller validation set can result in less reliable estimates of model performance. Validation accuracy may vary more from one run to another, making it harder to assess the model's generalization ability.There's a higher risk of overfitting because the model has more data to fit on during training, and we may not be able to detect overfitting effectively with a small validation set.

Ans 2

Data Representativeness:   The training set should be representative of the data distribution in our overall dataset. If our training set is too small, it may not capture the diversity and complexity of the data, leading to poor model generalization. A more representative training set can help our model learn patterns that are more likely to generalize to the test set.


Model Complexity:   The size of the training set can influence the complexity of the model we can effectively train. With a larger training set, we can potentially train more complex models without overfitting. However, if our training set is too small,we might be limited to simpler models that have fewer parameters to avoid overfitting.


Validation Set Size:   The size of the validation set is crucial for estimating how well our model will perform on unseen data (i.e., the test set). A larger validation set can provide a more reliable estimate of model performance because it tests the model on a larger portion of the data. This estimate can be used to make better predictions about how the model will perform on the test set.


Prediction Reliability:   If our validation set is very small relative to the training set, there's a risk that the validation set may not accurately represent the variability in the data. This can result in less reliable predictions of test set accuracy because the validation set may not effectively capture the full range of data scenarios the model will encounter during testing.

Ans 3


The percentage of data to reserve for the validation set depends on several factors, including the size of our overall dataset, the complexity of our model, and our specific goals. However, there are some common guidelines and rules of thumb that can help we strike a balance between having enough data for training and a representative validation set.


Here are a few commonly used splits:


70-30 Split (Training-Validation): This is a common starting point, especially for larger datasets. It reserves 30% of the data for validation. This split often provides a reasonable balance between model training and validation.


80-20 Split (Training-Validation): Another common split is 80% for training and 20% for validation. This provides a bit more data for training, which can be beneficial if our dataset is sufficiently large.


Cross-Validation: If our dataset is small, we might consider using techniques like k-fold cross-validation. In k-fold cross-validation, the data is divided into k subsets (or "folds"), and the model is trained and validated k times, with each fold serving as the validation set once. This approach helps we make efficient use of our limited data.


Stratified Sampling: If our dataset has class imbalances (e.g., some classes have significantly fewer samples than others), we may want to use stratified sampling to ensure that each class is represented proportionally in both the training and validation sets. Stratified sampling helps prevent issues where one class is underrepresented in the validation set.


Leave-One-Out Cross-Validation (LOOCV): In LOOCV, each data point serves as its own validation set, and the model is trained on the remaining data. This is useful when we have very limited data, but it can be computationally expensive for larger datasets.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?



Ans 1:


K-Fold Cross-Validation:
 One of the most widely used cross-validation techniques is k-fold cross-validation. In k-fold cross-validation, the data is divided into k subsets or "folds" of approximately equal size. The model is trained and evaluated k times, each time using a different fold as the validation set and the remaining folds for training. The validation accuracy for each fold is computed, and the results are averaged to obtain a more stable estimate of model performance.


Reduced Variability: Cross-validation helps reduce the impact of randomness in the initial data split. With a single random split,we might get a "lucky" or "unlucky" split that affects our assessment of model performance. By averaging results over multiple splits, we get a more robust estimate of how well our model is likely to perform on unseen data.


Better Generalization Assessment: Cross-validation provides a more comprehensive assessment of a model's generalization performance because it evaluates the model on multiple, non-overlapping subsets of the data. This is particularly useful when we have a limited dataset.


Model Selection: Cross-validation can also be used for model selection, helping we choose the best model or hyperparameters based on their average performance across multiple splits.

Ans 2 :


Cross-validation, specifically k-fold cross-validation, can provide a more accurate estimate of test accuracy compared to a single train-validation split when we have limited data. However, it's important to note that cross-validation estimates the model's performance on the validation (or holdout) data, not the true test data that the model has never seen before.

Ans 3 :

The number of iterations, often referred to as epochs in the context of training machine learning models, can have a significant effect on the estimate of model performance. However, whether more iterations result in a better estimate depends on the specific circumstances, and there is a trade-off to consider.

Here's how the number of iterations can impact the estimate of model performance:


Underfitting vs. Overfitting Trade-off:

When training a model, it goes through multiple iterations (epochs), adjusting its parameters to fit the training data better.
Initially, as the model learns from the data, its performance on both the training and validation sets tends to improve. This is because it's capturing the underlying patterns in the data.
However, if we train the model for too many epochs, it may start to overfit the training data, meaning it becomes too specialized in fitting the noise and specific examples in the training set. This can lead to a decrease in validation performance.


Early Stopping:

To obtain a good estimate of model performance, it's common practice to monitor the validation performance during training.
We can use techniques like early stopping to halt training when the validation performance starts to degrade, indicating the onset of overfitting. Early stopping helps prevent the model from learning noise in the data and provides a better estimate of its generalization performance.


Cross-Validation:

Cross-validation, particularly k-fold cross-validation, further mitigates the impact of the number of iterations on performance estimation. In k-fold cross-validation, the model is trained and evaluated multiple times, with different subsets of the data used for training and validation.
Cross-validation averages the performance over these different training-validation splits, providing a more robust estimate of how well the model generalizes, regardless of the number of training iterations.


More iterations do not necessarily lead to a better estimate of model performance. The key is to find the right balance:

Too few iterations may result in underfitting, where the model doesn't capture the underlying patterns in the data, leading to a poor estimate of performance.

Too many iterations can lead to overfitting, where the model learns noise in the training data, making the estimate overly optimistic.

Ans 4 :

Increasing the number of iterations  when dealing with a very small training dataset or validation dataset can help to a certain extent, but it may not completely compensate for the limitations imposed by the dataset's size. There are several considerations to keep in mind:

Model Overfitting: When we have a very small training dataset, increasing the number of iterations can potentially lead to overfitting. The model might start fitting the noise in the training data rather than capturing the underlying patterns. Overfitting can result in high training accuracy but poor generalization to new, unseen data, including the validation set or, more importantly, the test data.

Limited Data Diversity: A small dataset may not adequately represent the diversity and complexity of the real-world data that our model will encounter. No amount of additional training iterations can create information that is not present in the original data.

Early Stopping: To mitigate overfitting, it's essential to monitor the validation performance during training and employ early stopping. Early stopping involves halting training when the validation performance starts to degrade. While this technique can help control overfitting, it means that even with more iterations,we may not fully exploit the potential of our model.

Data Augmentation: If our dataset is very small, one way to make better use of it is through data augmentation techniques. Data augmentation involves applying transformations (e.g., rotations, translations, and flips for images) to create additional training examples from the existing ones. This can effectively increase the effective size of our training dataset without actually collecting more data.

Transfer Learning: In some cases, we may consider using pre-trained models or transfer learning. With transfer learning,we start with a model that has been pre-trained on a large dataset and fine-tune it on our smaller dataset. This leverages the knowledge already captured by the pre-trained model and can lead to better results with limited data.

 While increasing the number of iterations can help to some extent when dealing with a very small training dataset or validation dataset, it is not a magic solution. Careful consideration of techniques such as early stopping, data augmentation, and transfer learning is essential to make the most of limited data and improve model generalization. Additionally, it's crucial to set realistic expectations about the model's performance when working with small datasets.